In [13]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [14]:
## get the api keys
HF_TOKEN = os.getenv("HF_TOKEN")
OPENAI_API = os.getenv("OPENAI_API_KEY")

In [15]:
## hf login
login(HF_TOKEN, add_to_git_credential=True)

## openai client
openai_client=OpenAI(api_key=OPENAI_API)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [16]:
OPENAI_MODEL = 'gpt-4o-mini'
OPENSOURCE_MODEL = 'Qwen/CodeQwen1.5-7B-Chat'
OPENSOURCE_ENDPOINT = 'https://qejgyemly6rdpbl2.us-east4.gcp.endpoints.huggingface.cloud'

In [17]:
SYSTEM_PROMPT = """
You are an assistant that creates doc strings in reStructure Text format for an existing python function. 
Respond only with an updated python function; use comments sparingly and do not provide any explanation other than occasional comments.
Be sure to include typing annotation for each function argument or key word argument and return object types.
"""

In [18]:
def get_user_prompt(python_code):
    user_prompt = """
        Rewrite this Python function with doc strings in the reStructuredText style.
        Respond only with python code; do not explain your work other than a few comments.
        Be sure to write a description of the function purpose with typing for each argument and return.
        You are not allowed to change any part of the code.
        Take a look at the following code and work accordingly-
        """
    user_prompt+=python_code
    return user_prompt

In [19]:
def get_messages(python_code):
    return [
        {"role":"system", "content": SYSTEM_PROMPT},
        {"role":"user", "content": get_user_prompt(python_code)}
    ]

In [20]:
python_ = """
def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result


def calculate_2(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result
"""

In [21]:
def stream_gpt(python_code):
    output_stream = openai_client.chat.completions.create(
        model = OPENAI_MODEL,
        messages=get_messages(python_code),
        stream=True
    )
    reply=""
    for chunk in output_stream:
        text = chunk.choices[0].delta.content
        reply+=text or ""
        yield reply

In [22]:
## configuring OPENSOURCE model
def stream_open_source(python_code):
    tokenizer = AutoTokenizer.from_pretrained(OPENSOURCE_MODEL)
    messages = get_messages(python_code)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    client = InferenceClient(
        OPENSOURCE_ENDPOINT,
        token=HF_TOKEN
    )
    stream = client.text_generation(
        text,
        stream=True,
        details=True,
        max_new_tokens=2000
    )
    result = ""
    for r in stream:
        result+=r.token.text or ""
        yield result

In [23]:
def optimize(model, python_code):
    if model=="GPT":
        result = stream_gpt(python_code)
    else:
        result = stream_open_source(python_code)
    for r in result:
        yield r

In [24]:
with gr.Blocks() as ui:
    gr.Markdown("# Docstring generator")

    with gr.Row():
        python = gr.Textbox(label='Code without docstring', value = python_, lines=10)
        outcome = gr.Textbox(label="Code with docstring", lines=15)

    with gr.Row():
        model = gr.Dropdown(["GPT", "QWEN"], label="Select a Model", value="GPT")
    
    with gr.Row():
        generate = gr.Button("Generate Docstring")
    generate.click(fn=optimize, inputs=[model,python], outputs=[outcome])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
